## Naive Bayes

--------------------------------------------------------------------------------------------------------------------------------------

In [7]:
import glob, re
from typing import Set, NamedTuple, List, Tuple, Dict, Iterable
import math
from collections import defaultdict

Aqui, S é o evento "a mensagem é spam" e B é o evento "a mensagem contém a palavra bitcoin". Segundo o teorema de Bayes, a probabilidade de a mensagem ser spam condicionada a conter a palavra bitcoin é:

P(S|B) = [P(B|S)P(S)] / [P(B|S)P(S) + P(B|~S)P(~S)]

(proporção de mensagens sobre bitcoin que são spams)

Por exemplo, se 50% dos spams contêm a palavra bitcoin e apenas 1% dos não spams a contêm, a probabilidade de um e-mail contendo bitcoin ser spam é:

0.5 / (0.5 + 0.01) = 98%

#### Um filtro de spam mais sofisticado

In [4]:
# Construindo um classificador

def tokenize(text: str) -> Set[str]:
    text = text.lower()
    all_words = re.findall("[a-z0-9']+", text)
    return set(all_words)

assert tokenize("Data science is science") == {'data', 'science', 'is'}

In [6]:
class Message(NamedTuple):
    text: str
    is_spam: bool

In [14]:
class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5) -> None:
        self.k = k  # fator de suavização

        self.tokens: Set[str] = set()
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0

    # Método para treinar com várias mensagens
    def train(self, messages: Iterable[Message]) -> None:
        for message in messages:
            # Incrementando as contagens de mensagens
            if message.is_spam:
                self.spam_messages += 1
            else: 
                self.ham_messages += 1

            # Incrementando as contagens de palavras
            for token in tokenize(message.text):
                self.tokens.add(token)
                if message.is_spam:
                    self.token_spam_counts[token] += 1
                else:
                    self.token_ham_counts[token] += 1

    def _probabilities(self, token: str) -> Tuple[float, float]:
            """retorna P(token | spam) e P(token | not spam)"""
            spam = self.token_spam_counts[token]
            ham = self.token_ham_counts[token]

            p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
            p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)

            return p_token_spam, p_token_ham
    
    def predict(self, text: str) -> float:
        text_tokens = tokenize(text)
        log_prob_if_spam = log_prob_if_ham = 0.0

        # Iterando em cada palavra do vocabulário
        for token in self.tokens:
            prob_if_spam, prob_if_ham = self._probabilities(token)

            # se o *token* aparecer na mensagen, adicione o log da probabilidade de vê-lo
            if token in text_tokens:
                log_prob_if_spam += math.log(prob_if_spam)
                log_prob_if_ham += math.log(prob_if_ham)

            # Se não, adicione o log da probabilidade de _não_ vê-lo, que corresponde a log(1 - probabilidade de vê-lo)
            else:
                log_prob_if_spam += math.log(1.0 - prob_if_spam)
                log_prob_if_ham += math.log(1.0 - prob_if_ham)

        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)
        return prob_if_spam / (prob_if_spam + prob_if_ham)

#### Testando o classificador

In [15]:
messages = [Message("spam rules", is_spam=True),
            Message("ham rules", is_spam=False),
            Message("hello ham", is_spam=False)]

model = NaiveBayesClassifier(k=0.5)
model.train(messages)

In [16]:
# Verificando se as contagens estão corretas

assert model.tokens == {"spam", "ham", "rules", "hello"}
assert model.spam_messages == 1
assert model.ham_messages == 2
assert model.token_spam_counts == {"spam": 1, "rules": 1}
assert model.token_ham_counts == {"ham": 2, "rules": 1, "hello": 1}

In [21]:
# Fazendo uma previsão

text = "hello spam"

probs_if_spam = [
    (1 + 0.5) / (1 + 2 * 0.5),      # "spam"  (present)
    1 - (0 + 0.5) / (1 + 2 * 0.5),  # "ham"   (not present)
    1 - (1 + 0.5) / (1 + 2 * 0.5),  # "rules" (not present)
    (0 + 0.5) / (1 + 2 * 0.5)       # "hello" (present)
]

probs_if_ham = [
    (0 + 0.5) / (2 + 2 * 0.5),      # "spam"  (present)
    1 - (2 + 0.5) / (2 + 2 * 0.5),  # "ham"   (not present)
    1 - (1 + 0.5) / (2 + 2 * 0.5),  # "rules" (not present)
    (1 + 0.5) / (2 + 2 * 0.5),      # "hello" (present)
]

p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam))
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))

# Deve ser aproximadamente 0.83
print(model.predict(text))

0.8350515463917526
